<a href="https://colab.research.google.com/github/YKG-123/Fantasy_football_ML_Random_forest_Regressor/blob/main/WR_preseason_postable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
drive.mount('/content/drive')
WR_2025=pd.read_csv('/content/drive/MyDrive/WR_2025.csv')
WR_2025.rename(columns={"adp":"adp_for_model"})
WR_2025.rename(columns={"adp_for_rank": "adp_true"}, inplace=True)
WR_2025=WR_2025.reset_index(drop=True)
WR_2025.rename(columns={"adp_true": "adp"}, inplace=True)

Mounted at /content/drive


In [ ]:

# --- 1. Ensure column uniqueness ---
WR_2025 = WR_2025.loc[:, ~WR_2025.columns.duplicated()]

# --- 2. Fill missing ADP with sentinel ---
WR_2025["adp"] = WR_2025["adp"].fillna(200)

# --- 3. Filter out undrafted ---
WR_2025 = WR_2025[WR_2025["adp"] != 200]  # keep only drafted players

# --- 4. Compute ranks ---
WR_2025["predicted_rank"] = WR_2025["predicted_target_season"].rank(ascending=False)
WR_2025["adp_rank"]       = WR_2025["adp"].rank(ascending=True)

# --- 5. ADP minus model prediction gap ---
WR_2025["adp-predictions"] = WR_2025["adp_rank"] - WR_2025["predicted_rank"]

# --- 6. Sort so biggest difference appears first ---
WR_2025 = WR_2025.sort_values(by="predicted_rank", ascending=True).reset_index(drop=True)
WR_2025["adp"] = WR_2025["adp"].round(2)
# --- 7. Preview top 75 WRs ---
WR_2025[["player_name", "adp", "predicted_rank", "adp_rank", "adp-predictions"]].head(100)


,player_name,adp,predicted_rank,adp_rank,adp-predictions
0,Amon-Ra St. Brown,13.0,1.0,7.0,6.0
1,A.J. Brown,19.2,2.0,10.0,8.0
2,Ja'Marr Chase,1.5,3.0,1.0,-2.0
3,Nico Collins,11.1,4.0,6.0,2.0
4,Justin Jefferson,4.9,5.0,2.0,-3.0
...,...,...,...,...,...
75,Ray-Ray McCloud,166.8,76.0,73.0,-3.0
76,Darius Slayton,170.2,77.0,77.0,0.0
77,Michael Wilson,164.3,78.0,70.0,-8.0
78,Calvin Austin,163.3,79.0,69.0,-10.0
